# This workbook uses for loop to iterate over SQL generated list of URLs based on URL pattern convention

## Import Packages

In [ ]:
import requests                #Requests is an elegant and simple HTTP library for Python, built for human beings.
from bs4 import BeautifulSoup  #Beautiful Soup is a Python library for pulling data out of HTML and XML files. It works with your favorite parser to provide idiomatic ways of navigating, searching, and modifying the parse tree. It commonly saves programmers hours or days of work.
import pandas as pd            #pandas is a fast, powerful, flexible and easy to use open source data analysis and manipulation tool, built on top of the Python programming language.
import numpy as np             #NumPy is a library for the Python programming language, adding support for large, multi-dimensional arrays and matrices, along with a large collection of high-level mathematical functions to operate on these arrays
import os                      #This module provides a portable way of using operating system dependent functionality.

In [ ]:
os.getcwd()

## File List

In [ ]:
url_file = r'C:\Users\halin\OneDrive\Documents\Data_Coding_Reporting\Inguz\URLs_1.1.20_to_12.31.22.xlsx'
df_url = pd.read_excel(url_file,engine='openpyxl')
url_list = df_url['Full URL']


## HTML Parsing, Looping, and Appending

In [ ]:
data = []                                         #create list to append each dataframe interation to

for url in url_list[:798]:                          #slicing takes 0 to 2 (798 will return data through 3.9.22)
    page = requests.get(url)
    soup = BeautifulSoup(page.content, "html.parser")
    results = soup.find('div',id='left-area')
    date_results = soup.find(class_ = 'entry-title')
    workout_elements = results.find_all('p',class_ = lambda x: x != 'wp-caption-text' and x != 'post-meta', style=False)
    #print(results.prettify())
    
    d = []

    for workout_element in workout_elements:
        d.append(
            {
                workout_element.text.strip()
            })
    
    df = pd.DataFrame(d)
    df.columns=['Data']
    df['Data'] = df['Data'].str.split('\n')
    df2= df.explode('Data',ignore_index=True)
    if date_results is not None:
        date_text = date_results.text
    else: 
        date_text = None 
    df2['Date'] = date_text
    df2['Row Type'] = np.nan
    df2.loc[df2['Data'].str.contains('STRENGTH', case=False)==True,'Row Type'] = 'STRENGTH'
    df2.loc[df2['Data'].str.contains('WOD', case=False)==True,'Row Type'] = 'WOD'
    df2.loc[df2['Data'].str.contains('MOVE 45', case=False)==True,'Row Type'] = 'Move 45'
    df2.loc[df2['Data'].str.contains('ACCESSORY WORK', case=False)==True,'Row Type'] = 'ACCESSORY WORK'
    df2.loc[df2['Data'].str.contains('After Party', case=False)==True,'Row Type'] = 'After Party'
    df2.loc[df2['Data'].str.contains('Olympic Lifting', case=False)==True,'Row Type'] = 'Olympic Lifting'
    df2['Row Type']=df2['Row Type'].ffill()
    df_wod = df2[df2['Row Type']=='WOD']
    data.append(df_wod)
    
appended_data=pd.concat(data)
appended_data.to_csv('wod_data.csv',index=False)

In [ ]:
df_wod #validating DataFrame

In [ ]:
appended_data #validating DataFrame